In [10]:
import json
import nltk
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from torch.nn.utils import clip_grad_norm_
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer

from gensim.models import Word2Vec

In [11]:
torch.manual_seed(1)

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Mihaela
[nltk_data]     Stoycheva\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load data

In [13]:
def load_data(filename, max_sentence_len):
    # the tokenizer splits <unk> so we use MWETokenizer to re-merge it
    data_original = []
    data_padded = []
    with open(filename) as f:
        for line in f:
            sentence, padded_sentence = tokenize_sentence(line, max_sentence_len)
            data_original.append(sentence)
            data_padded.append(padded_sentence)
    
    return data_original, data_padded

def tokenize_sentence(string, max_sentence_len):
    merger = MWETokenizer([('<', 'unk', '>')], separator = '') 
    sentence = word_tokenize(string.strip())       # tokenize sentence
    sentence = merger.tokenize(sentence)         # merge <unk>
    sentence = sentence[:max_sentence_len - 2]   # cut sentence at max_sentence_length
    sentence = ['<sos>'] + sentence + ['<eos>']  # add start and end-of-sentence tags

    # pad the rest of the sentence
    padded_sentence = sentence.copy()
    padded_sentence.extend(['<pad>']*(max_sentence_len - len(sentence))) 
    
    return sentence, padded_sentence

In [14]:
max_sentence_length = 50
train_data, train_data_padded = load_data("data/ptb.train.txt", max_sentence_length)
val_data, val_data_padded = load_data("data/ptb.valid.txt", max_sentence_length)
test_data, test_data_padded = load_data("data/ptb.test.txt", max_sentence_length)

### Create Word2Vec word embeddings

In [15]:
embedding_size = 500
num_epochs_to_train = 100

In [16]:
word2vec_model = Word2Vec(train_data, min_count=1, size=embedding_size, window=5)
word2vec_model.train(train_data, epochs=num_epochs_to_train, total_examples=word2vec_model.corpus_count)
# word_embeddings.train(train_data, len(train_data), epochs=100, total_examples=word_embeddings.corpus_count)

(66651077, 96960000)

In [17]:
word2vec_model.wv.most_similar("stocks")
# word2vec_model.wv['credit']

[('issues', 0.3977467715740204),
 ('stock', 0.37520620226860046),
 ('traders', 0.3692500591278076),
 ('cds', 0.3398664593696594),
 ('investors', 0.3379707336425781),
 ('companies', 0.31781235337257385),
 ('indexes', 0.3151756227016449),
 ('yields', 0.3063001036643982),
 ('nasdaq', 0.3056687116622925),
 ('currencies', 0.30312639474868774)]

In [18]:
vocabulary_size = len(word2vec_model.wv.vocab)
print("size of the vocabulary:", vocabulary_size)

size of the vocabulary: 10005


### Split data into batches

In [19]:
def get_batches(data, data_padded, batch_size, pad_index, word2vec_model):
    inputs = []
    targets = []
    lengths = []
    for i in range(len(data) // batch_size):
        # take batch_size sentences from the data each time
        batch_sentences = data[i*batch_size:(i+1)*batch_size]
        batch_sentence_lens = [len(x) for x in batch_sentences]
        
        # sentences in a batch have to be sorted in decreasing order of length (for pack_padded_sentence)
        sorted_pairs = sorted(zip(batch_sentence_lens,batch_sentences), reverse=True)
        batch_sentences = [sentence for length, sentence in sorted_pairs]
        batch_sentence_lens = [length-1 for length, sentence in sorted_pairs]
        
        # each input and target is a (batch_size x max_sentence_len-1 x 1) matrix
        # initially filled with the index for padditng tag <pad>
        input_batch = np.ones((batch_size, len(data_padded[0])-1, 1)) * pad_index
        target_batch = np.ones((batch_size, len(data_padded[0])-1, 1)) * pad_index
        
        # for each sentence in the batch, fill the corresponding row in current_batch
        # with the indexed of the words in the sentence (except for <pad>)
        for j, sentence in enumerate(batch_sentences):
            word_indexes = np.array([word2vec_model.wv.vocab[word].index for word in sentence])
            input_batch[j,0:len(sentence)-1,0] = word_indexes[:-1]
            target_batch[j,0:len(sentence)-1,0] = word_indexes[1:]
        
        # make the matrices into torch tensors and append
#         inputs.append(torch.tensor(input_batch))
#         targets.append(torch.tensor(target_batch))
        inputs.append(input_batch)
        targets.append(target_batch)
        lengths.append(batch_sentence_lens)
    return inputs, targets, lengths

### Define RNNLM

In [20]:
class RNNLM(nn.Module):
    def __init__(self, vocabulary_size, embedding_size, hidden_size, num_layers, embedding_weights):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding.from_pretrained(embedding_weights)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocabulary_size)
        self.embedding_size = embedding_size
        
    def forward(self, x, hidden, x_lens, train=True):
        batch_size, max_len, _ = x.shape
        embedding_dim = self.embedding_size

        x = self.embed(torch.tensor(x, dtype=torch.long)).view(batch_size, max_len, embedding_dim)
        if train:
            x_lens = torch.tensor(x_lens, dtype=torch.long)
            x = pack_padded_sequence(x, x_lens, batch_first=True)

        out, hidden = self.lstm(x.float(), hidden) 
        
        if train:
            out, output_lens = pad_packed_sequence(out, batch_first=True, total_length=max_sentence_length-1)

        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        out = self.linear(out)
        
        return out, hidden

### Train RNNLM

In [21]:
input_size = 100
output_size = 100
hidden_size = 50

In [22]:
batch_size = 20
use_first_k = 500
padding_index = vocabulary_size
train_batches, train_targets, train_sentence_lens = get_batches(train_data[:use_first_k], train_data_padded[:use_first_k], 
                                                                batch_size, padding_index, word2vec_model)

In [23]:
# make the word embeddings into a pythorch tensor
embedding_weights = word2vec_model.wv.vectors
embedding_weights = np.vstack((embedding_weights, np.zeros((1,embedding_size))))  # add zero vector for <pad>
embedding_weights = torch.tensor(embedding_weights)

In [24]:
learning_rate = 0.001
num_layers = 1
epochs = 10

model = RNNLM(vocabulary_size, embedding_size, hidden_size, num_layers, embedding_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
for epoch in range(epochs):
    hidden = (torch.zeros(num_layers, batch_size, hidden_size), torch.zeros(num_layers, batch_size, hidden_size))
    for i in range(len(train_batches)):
        x = train_batches[i]
        x_lens = train_sentence_lens[i]
        y = torch.tensor(train_targets[i].reshape(-1), dtype=torch.long)   
        h, c = hidden
        h = h.detach()
        c = c.detach()
        hidden = (h, c)
    
        outputs, hidden = model(x, hidden, x_lens)
        
        mask = (y < padding_index)
        loss = nn.CrossEntropyLoss()(outputs[mask], y[mask])

        model.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

    if epoch % 1 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
               .format(epoch + 1, epochs, loss.item(), np.exp(loss.item())))

Epoch [1/10], Loss: 8.9415, Perplexity: 7642.30
Epoch [2/10], Loss: 7.5489, Perplexity: 1898.71
Epoch [3/10], Loss: 6.4521, Perplexity: 634.03
Epoch [4/10], Loss: 6.2294, Perplexity: 507.45
Epoch [5/10], Loss: 6.1694, Perplexity: 477.88
Epoch [6/10], Loss: 6.1256, Perplexity: 457.41
Epoch [7/10], Loss: 6.0735, Perplexity: 434.21
Epoch [8/10], Loss: 6.0234, Perplexity: 412.99
Epoch [9/10], Loss: 5.9735, Perplexity: 392.88
Epoch [10/10], Loss: 5.9201, Perplexity: 372.45


In [26]:
# Predict
test_sentences = ["he could see"]
sentence, _ = tokenize_sentence(test_sentences[0], max_sentence_length)
sentence = sentence[:-1]
word_indexes = np.array([word2vec_model.wv.vocab[word].index for word in sentence]).reshape(1, len(sentence), 1)
print(word_indexes.shape)

hidden = (torch.zeros(1, 1, hidden_size), torch.zeros(1, 1, hidden_size))
h, c = hidden
h = h.detach()
c = c.detach()
hidden = (h, c)

outputs, hidden = model(word_indexes, hidden, x_lens, train=False)
softmax_outputs = F.softmax(outputs, dim=1).detach().numpy()
last_word = softmax_outputs[-1,:]
predicted_next_word_idx = np.random.choice(range(len(last_word)), p=last_word)
print("Argmax: ", word2vec_model.wv.index2word[np.argmax(last_word)])
print("Next word: ", word2vec_model.wv.index2word[predicted_next_word_idx])

(1, 4, 1)
Argmax:  the
Next word:  previously


In [42]:
# THIS CAN BE DELETED?
# encoder = Encoder(embedding_size, hidden_size, num_layers, embedding_weights)
# encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)

# decoder = Decoder(vocabulary_size, embedding_size, hidden_size, num_layers, embedding_weights)
# decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)

In [27]:
# THIS CAN BE DELETED?
# for epoch in range(epochs):
#     for i in range(len(train_batches)):
#         x = train_batches[i]
#         x_lens = train_sentence_lens[i]
#         y = torch.tensor(train_targets[i].reshape(-1), dtype=torch.long)
    
#         hidden = encoder.init_hidden(batch_size)
        
#         _, hidden = encoder(x, hidden, x_lens)
#         outputs, hidden = decoder(x, hidden, x_lens)
        
#         mask = (y < padding_index)
#         loss = nn.CrossEntropyLoss()(outputs[mask], y[mask])

#         encoder.zero_grad()
#         decoder.zero_grad()
#         loss.backward()
#         clip_grad_norm_(encoder.parameters(), 0.5)
#         clip_grad_norm_(decoder.parameters(), 0.5)
#         encoder_optimizer.step()
#         decoder_optimizer.step()

#     if epoch % 1 == 0:
#         print ('Epoch [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
#                .format(epoch + 1, epochs, loss.item(), np.exp(loss.item())))

NameError: name 'encoder' is not defined

In [28]:
class Encoder(nn.Module):
    def __init__(self, hidden_size, num_layers, embedding_weights):
        super(Encoder, self).__init__()
        # parameters
        self.embedding_size = embedding_weights.shape[1]
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_first = True
        
        #layers
        self.embed = nn.Embedding.from_pretrained(embedding_weights)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, batch_first=self.batch_first)
        
    def forward(self, x, hidden, x_lens, train=True):
        batch_size, max_len, _ = x.shape
        
        x = torch.tensor(x, dtype=torch.long)  # make the input into a torch tensor
        x = self.embed(x).view(batch_size, max_len, self.embedding_size)

        if train:
            x_lens = torch.tensor(x_lens, dtype=torch.long)
            x = pack_padded_sequence(x, x_lens, batch_first=self.batch_first)
            
        output, hidden = self.lstm(x.float(), hidden) 

        if train:
            output, output_lens = pad_packed_sequence(output, batch_first=self.batch_first, 
                                                      total_length=max_sentence_length-1)
        
#         output = output.reshape(output.size(0)*output.size(1), output.size(2))
#         output = self.linear(output)
        
        return output, hidden
    
    def init_hidden(self, batch_size):
        h = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return (h, c)

In [29]:
class Decoder(nn.Module):
    def __init__(self,hidden_size, num_layers, embedding_weights):
        super(Decoder, self).__init__()
        # parameters
        self.vocabulary_size = embedding_weights.shape[0]
        self.embedding_size = embedding_weights.shape[1]
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_first = True
        
        # layers
        self.embed = nn.Embedding.from_pretrained(embedding_weights)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, batch_first=self.batch_first)
        self.linear = nn.Linear(self.hidden_size, self.vocabulary_size)
#         self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden, x_lens, train=True):
        batch_size, max_len, _ = x.shape
        
        x = torch.tensor(x, dtype=torch.long)  # make the input into a torch tensor
        x = self.embed(x).view(batch_size, max_len, self.embedding_size)
        
        if train:
            x_lens = torch.tensor(x_lens, dtype=torch.long)
            x = pack_padded_sequence(x, x_lens, batch_first=self.batch_first)

        output, hidden = self.lstm(x.float(), hidden) 
        
        if train:
            output, output_lens = pad_packed_sequence(output, batch_first=self.batch_first, 
                                                      total_length=max_sentence_length-1)
        
        output = output.reshape(output.size(0)*output.size(1), output.size(2))
        output = self.linear(output)
        
        return output, hidden

In [34]:
class VAE(nn.Module):
    def __init__(self, hidden_dim, num_layers, embedding_weights, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(hidden_size, num_layers, embedding_weights)
        self.decoder = Decoder(hidden_size, num_layers, embedding_weights)
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.batch_first = True
        
        self.hidden_to_mean = nn.Linear(2 * self.hidden_dim * num_layers, self.latent_dim, self.batch_first)
        self.hidden_to_logvar = nn.Linear(2 * self.hidden_dim * num_layers, self.latent_dim, self.batch_first)
        self.latent_to_hidden = nn.Linear(latent_dim, 2 * self.hidden_dim * num_layers, self.batch_first)
        
    def reparametrize(self, mean, log_variance):
        eps = torch.randn_like(mean)
        return mean + eps * torch.exp(0.5 * log_variance)
        
    def encode(self, x, x_lens):
        batch_size, max_len, _ = x.shape
        hidden = self.encoder.init_hidden(batch_size)
        _, hidden = self.encoder.forward(x, hidden, x_lens)
        return hidden
        
    def decode(self, hidden, x, x_lens):
        outputs, _ = self.decoder.forward(x, hidden, x_lens)
        return outputs
    
    def forward(self, x, x_lens):
        hidden = self.encode(x, x_lens)
        hidden_concatenated = torch.cat((hidden[0], hidden[1]), 2)
        
        mean = self.hidden_to_mean(hidden_concatenated)
        
        log_variance = self.hidden_to_logvar(hidden_concatenated)
        z = self.reparametrize(mean, log_variance)
        hidden = self.latent_to_hidden(z)
        
        hidden = torch.split(hidden, self.hidden_dim, dim=2)
        
        outputs = self.decode(hidden, x, x_lens)
        return mean, log_variance, outputs

In [37]:
def loss_function(outputs, labels, mean, log_variance):
    BCE = nn.CrossEntropyLoss()(outputs, labels)
    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + log_variance - mean.pow(2) - log_variance.exp())

    return BCE + KLD
                           
vae = VAE(hidden_size, num_layers, embedding_weights, 10)
vae_optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)
for epoch in range(epochs):
    for i in range(len(train_batches)):
        x = train_batches[i]
        x_lens = train_sentence_lens[i]
        y = torch.tensor(train_targets[i].reshape(-1), dtype=torch.long)
    
        mean, log_variance, outputs = vae(x, x_lens)
        
        mask = (y < padding_index)
        #loss = nn.CrossEntropyLoss()(outputs[mask], y[mask]) # TODO CHANGE THIS TO VAE LOSS
        loss = loss_function(outputs[mask], y[mask], mean, log_variance)

        vae.zero_grad()
        loss.backward()
        clip_grad_norm_(vae.parameters(), 0.5)
        vae_optimizer.step()

    if epoch % 1 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
               .format(epoch + 1, epochs, loss.item(), np.exp(loss.item())))

Epoch [1/10], Loss: 12.0595, Perplexity: 172739.80
Epoch [2/10], Loss: 8.3299, Perplexity: 4145.82
Epoch [3/10], Loss: 6.7429, Perplexity: 848.04
Epoch [4/10], Loss: 6.4913, Perplexity: 659.35
Epoch [5/10], Loss: 6.3347, Perplexity: 563.81
Epoch [6/10], Loss: 6.1928, Perplexity: 489.21
Epoch [7/10], Loss: 6.0860, Perplexity: 439.66
Epoch [8/10], Loss: 5.9729, Perplexity: 392.63
Epoch [9/10], Loss: 5.8710, Perplexity: 354.60
Epoch [10/10], Loss: 5.7646, Perplexity: 318.80


In [124]:
# TODO WRITE THE VAE LOSS

# def VAE_loss(outputs, labels, nz, mean, log_variance):
#     local_mu = K.repeat(mu, k)
#     local_sigma = K.repeat(sigma, k)
#     log_posterior = -(n_z / 2) * log2pi - K.sum(K.log(1e-8 + local_sigma) + 0.5 * K.square(z - local_mu) / K.square(1e-8 + local_sigma), axis=-1)
#     log_prior = -(n_z / 2) * log2pi - K.sum(0.5 * K.square(z), axis=-1)
#     log_bernoulli = K.sum(y_true * K.log(y_pred + 1e-8) + (1 - y_true) * K.log(1 - y_pred + 1e-8), axis=-1)
#     log_weights = log_bernoulli + log_prior - log_posterior
#     return -K.mean(log_weights, axis=-1)

In [39]:
# Predict
test_sentences = ["he could see"]
sentence, _ = tokenize_sentence(test_sentences[0], max_sentence_length)
sentence = sentence[:-1]
word_indexes = np.array([word2vec_model.wv.vocab[word].index for word in sentence]).reshape(1, len(sentence), 1)
print(word_indexes.shape)

hidden = (torch.zeros(1, 1, hidden_size), torch.zeros(1, 1, hidden_size))
h, c = hidden
h = h.detach()
c = c.detach()
hidden = (h, c)

outputs, hidden = model(word_indexes, hidden, x_lens, train=False)
softmax_outputs = F.softmax(outputs, dim=1).detach().numpy()
last_word = softmax_outputs[-1,:]
predicted_next_word_idx = np.random.choice(range(len(last_word)), p=last_word)
print("Argmax: ", word2vec_model.wv.index2word[np.argmax(last_word)])
print("Next word: ", word2vec_model.wv.index2word[predicted_next_word_idx])

(1, 4, 1)
Argmax:  the
Next word:  company
